In [ ]:
In this notebook, we would test different batch contraction model include loop, batch, sparse batch

# Batch Method

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

In [1]:
import numpy as np
import torch
import time
from tqdm.notebook import trange, tqdm
from utils import *

D=10
P=4
L=14
def rd_engine(*x,**kargs):
    x =  torch.randn(*x,device='cpu',**kargs)
    x/=  torch.norm(x).sqrt()
    return x
def generate_test_data(L=10,num=20,k=2):
    #images,labels = iter(train_loader).next()
    #inputs = preprocess_sum_one(images)
    inputs = rd_engine(L,num,k)
    inputs = inputs.permute(1,2,0)#(B,num,k)->(num,k,B)
    inputs = torch.diag_embed(inputs)#(num,k,B)->(num,k,B,B)
    inputs = inputs.permute(0,2,1,3)#(num,k,B,B)->(num,B,k,B)
    #inputs= [v for v in inputs]
    #inputs[0]= torch.diagonal(inputs[0], dim1=0, dim2=-1)#(B,k,B) -> #(k,B)
    return inputs


In [170]:
from torchvision import datasets, transforms
mnist_data = np.load('archive/tn-for-unsup-ml/data/binarized_mnist.npz')
train_data = torch.from_numpy(mnist_data['train_data'])
test_data  = torch.from_numpy(mnist_data['test_data'])

transform = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Normalize(mean=(0.0,), std=(1.0,))
])
DATAPATH    = '/media/tianning/DATA/DATASET/MNIST/'
mnist_train = datasets.MNIST(DATAPATH, train=True, download=False, transform=transform)
mnist_test  = datasets.MNIST(DATAPATH, train=False,download=False, transform=transform)
train_loader= torch.utils.data.DataLoader(dataset=mnist_train, batch_size=1000, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=mnist_test , batch_size=1000, shuffle=False)
images,labels = iter(train_loader).next()

In [130]:
now_result = np.einsum("abc,cd,de,ef,fg,gh,hi,ijk->abjk",
                       imps.get_tensor(len(imps) - 1), 
                       inv_sqrtl, 
                       U, 
                       np.sqrt(lam),np.sqrt(lam), 
                       V, 
                       inv_sqrtr, 
                       imps.tensors[0])

###### torch.dense

In [11]:
a,X,U,S,V,Y,b = canonicalize(imps)

In [17]:
from engine.torch_dense import *
from tqdm.notebook import trange, tqdm

In [54]:
left  = np.einsum('ea,ab,bc->ec',X,U,np.sqrt(S)).real
right = np.einsum('ea,ab,bc->ec',np.sqrt(S),V,Y).real

In [60]:
np.einsum('ab,ca->bc',X,Y).round(3)

array([[ 1.35 +0.j, -1.134+0.j,  0.117+0.j,  2.915+0.j],
       [ 1.193+0.j,  1.376+0.j, -3.855+0.j,  0.129+0.j],
       [-1.92 +0.j,  2.322+0.j,  0.48 +0.j,  0.344+0.j],
       [ 2.836+0.j,  0.947+0.j,  0.577+0.j, -0.068+0.j]])

###### diagonal.sparse

In [2]:
from engine.sparse import *
from tqdm.notebook import trange, tqdm

In [134]:
imps = FiniteMPS.random(d=[3,3]*5,
      D=[2]*9,
      dtype=np.float64)
imps.canonicalize()
imps.position(2)

1.0

In [4]:
L=10
idx1     = list(range(2*L))
idx2     = [i for i in range(L) for j in range(2)]
mps_unit = sparse.COO([idx1,idx2],np.random.randn(2*L),(2*L,L)).reshape((L,L*2))
R,Q = diagonal_tensor_RQ(mps_unit)
print(R.shape)
print(Q.shape)
# u0,s0,v0 = diagonal_tensor_svd_sparse_2D(mps_unit)
# u1,s1,v1 = np.linalg.svd(mps_unit.todense(),full_matrices=False)

(10, 10)
(10, 20)


In [158]:
a=np.random.randint(3,(100,100)).astype('uint8')

In [161]:
np.save("tttest",a)

In [162]:
b=np.load("tttest.npy")

In [166]:
b

array([16, 43], dtype=uint8)

In [144]:
class Efficient_Sparse_Matrix_List_Saver:
    def __init__(self,dtype = 'sparse'):
        self.dtype = dtype
    def save(self,sparse_matrix_list,save_dir):
        if self.dtype == 'sparse':
            self.save_sparse_data(sparse_matrix_list,save_dir)
        else:
            raise NotImplementedError
    def load(self,save_dir):
        if self.dtype == 'sparse':
            return self.load_sparse_data(save_dir)
        else:
            raise NotImplementedError
    @staticmethod 
    def save_sparse_data(sparse_matrix_list,save_dir):
        max_shape_len = max([len(t.shape) for t in sparse_matrix_list])
        save_indexes  = []
        save_shapes   = []
        for i in range(len(sparse_matrix_list)):
            save_index = sparse_matrix_list[i].coords.transpose()
            save_shape = list(sparse_matrix_list[i].shape)
            if len(save_shape)< max_shape_len:
                padding     = max_shape_len-len(save_shape)
                save_index  = np.pad(save_index,[[0,0],[0,padding]])
                save_shape  = save_shape+[1]*padding
            save_indexes.append(save_index) 
            save_shapes.append(save_shape)
        all_indexs  = np.concatenate(save_indexes)
        all_values  = np.concatenate([t.data for t in sparse_matrix_list])
        all_shape   = np.stack(save_shapes)
        all_idx_size= np.array([len(t.shape) for t in sparse_matrix_list])
        nnz_list    = np.array([t.nnz for t in sparse_matrix_list])
        assert sum(nnz_list) == len(all_indexs) == len(all_values)
        np.save(os.path.join(save_dir,"all_indexs"),all_indexs)
        np.save(os.path.join(save_dir,"all_idx_size"),all_idx_size)
        np.save(os.path.join(save_dir,"all_values"),all_values)
        np.save(os.path.join(save_dir,"all_shape"),all_shape)
        np.save(os.path.join(save_dir,"nnz_list"),nnz_list)
    @staticmethod
    def load_sparse_data(save_dir):
        all_indexs   = np.load(os.path.join(save_dir,"all_indexs.npy"))
        all_idx_size = np.load(os.path.join(save_dir,"all_idx_size.npy"))
        all_values   = np.load(os.path.join(save_dir,"all_values.npy"))
        all_shape    = np.load(os.path.join(save_dir,"all_shape.npy"))
        nnz_list     = np.load(os.path.join(save_dir,"nnz_list.npy"))
        
        sparse_matrix_list =[]
        start = 0 
        for nnz,sz,shape in zip(nnz_list,all_idx_size,all_shape):
            indexs = all_indexs[start:start+nnz][...,:sz].transpose()
            values = all_values[start:start+nnz]
            shape  = shape[:sz]
            tensor = sparse.COO(indexs,values,shape.tolist())
            start  = start+nnz
            sparse_matrix_list.append(tensor)
        return sparse_matrix_list

In [56]:
L=10
idx1     = list(range(2*L))
idx2     = [i for i in range(L) for j in range(2)]
#mps_unit = sparse.COO([idx1,idx2],np.random.randn(2*L),(2*L,L)).reshape((L,2,L))
mps_line = ([sparse.as_coo(np.random.randn(2,L))]+
            [sparse.COO([idx1,idx2],np.random.randn(2*L),(2*L,L)).reshape((L,2,L))
               for i in range(9)])
#mps_line[0]

In [34]:
train_loader= torch.utils.data.DataLoader(dataset=mnist_train, batch_size=1000, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=mnist_test , batch_size=1000, shuffle=False)
images,labels = iter(train_loader).next()
origin_inputs = preprocess_sum_one(images)

In [35]:
L=origin_inputs.shape[0]
idx1     = list(range(2*L))
idx2     = [i for i in range(L) for j in range(2)]
mps_line=[sparse.as_coo(origin_inputs[:,0,:].transpose(1,0).numpy())]
for tensor in origin_inputs.permute(1,0,2)[1:]:
    #print(tensor.flatten().numpy().shape)
    mps_line.append(sparse.COO([idx1,idx2],tensor.flatten().numpy(),(2*L,L)).reshape((L,2,L)))

In [36]:
# next_line,z = right_canonicalize_MPS_sparse(mps_line,final_normlization =True,all_renormlization=False)

# Decomposition_Engine=lambda x:truncated_SVD_sparse(x,output='QR',
#                                                     max_truncation_error=0.00,
#                                                     max_singular_values=100,
#                                                    )
# next_line_2,z_2 = left_canonicalize_MPS_sparse(next_line,final_normlization =True,
#                                                all_renormlization=True,
#                                                Decomposition_Engine=Decomposition_Engine)

In [54]:
%%timeit 
u,s,v = diagonal_tensor_svd_sparse(sparse_inputs)

8.67 ms ± 33.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


###### diagonal.torch.dense

In [58]:
def diagonal_tensor_svd_torch_dense(tensor):
    reduce = False
    if len(tensor.shape)==2:
        tensor = tensor.unsqueeze(0)
        reduce = True
    W,H   = tensor.shape[-2:]
    batch_shape= tensor.shape[:-2]
    u = s = v = None
    if W>=H:
        batch_diag = torch.matmul(tensor.transpose(-1,-2),tensor)#auto broadcast, or can use bmm
    else:
        batch_diag = torch.matmul(tensor,tensor.transpose(-1,-2))#auto broadcast, or can use bmm
    A,A        = batch_diag.shape[-2:]
    batch_diag = batch_diag[...,range(A),range(A)]
    batch_diag,batch_order= batch_diag.sort(-1,descending=True)
    #fast_V      = [get_sort_matrix(order).to_dense() for order in batch_order]
    batch_order = batch_order.flatten(start_dim=0,end_dim=-2)
    K,A         = batch_order.shape
    s           = batch_diag.sqrt()
    s           = s.reshape(*batch_shape,A)
    if W>=H:
        v = torch.sparse_coo_tensor([list(range(K*A)),batch_order.flatten().tolist()], [1.0]*K*A,(K*A,A))
        v = v.to_dense().reshape(-1,A,A)
        u = torch.bmm(tensor,v.transpose(-1,-2)/s.unsqueeze(-2))
        v = v.reshape(*batch_shape,A,A)
        u = u.reshape(*batch_shape,W,A)
    else:
        u = torch.sparse_coo_tensor([list(range(K*A)),batch_order.flatten().tolist()], [1.0]*K*A,(K*A,A))
        u = u.to_dense().reshape(-1,A,A).transpose(-1,-2)
        v = torch.bmm(u.transpose(-1,-2)/s.unsqueeze(-1),tensor)#TODO: case when s==0
        u = u.reshape(*batch_shape,A,A)
        v = v.reshape(*batch_shape,A,H)
    output = [u,s,v.transpose(-1,-2)]
    if reduce:
        output = [t[0] for t in output]
    return output

In [10]:
# i=0
# for images,labels in tqdm(train_loader):
#     #images,labels = iter(train_loader).next()
#     inputs = preprocess_sum_one(images)
#     #inputs= rd_engine(10,50,2)
#     inputs= inputs.permute(1,2,0)#(B,num,k)->(num,k,B)
#     inputs= torch.diag_embed(inputs)#(num,k,B)->(num,k,B,B)
#     inputs= inputs.permute(0,2,1,3)#(num,k,B,B)->(num,B,k,B)
#     inputs= [v for v in inputs]
#     inputs[0]= torch.diagonal(inputs[0], dim1=0, dim2=-1)#(B,k,B) -> #(k,B)
#     DCEngine = lambda x:truncated_SVD(x,output='QR',max_truncation_error=0.00,max_singular_values=100)
#     mps_line,Z_list = left_canonicalize_MPS(inputs,Decomposition_Engine=DCEngine,normlization=False)
#     state_dict = {}
#     state_dict['xdata']=dict([[i,t] for i,t in enumerate(mps_line)])
#     state_dict['ydata']=labels
#     torch.save(state_dict,f'offline_SVD_data/preprocess_sum_one.cut100/mps_line_{i}.pt')
#     i+=1

  0%|          | 0/60 [00:00<?, ?it/s]

In [62]:
Q,R        = Decomposition_Engine(inputs[0])[:2]
new_tensor = torch.einsum('ab,bcd->acd',R,inputs[1])
shape      = new_tensor.shape
a,b,c = shape
new_tensor = new_tensor.reshape(a*b,c)

use 1/2 sing vals


###### diagonal.torch_sparse

In [ ]:
# L=10
# idx1     = list(range(2*L))
# idx2     = [i for i in range(L) for j in range(2)]
# #mps_unit = sparse.COO([idx1,idx2],np.random.randn(2*L),(2*L,L)).reshape((L,2,L))
# mps_line = ([sparse.as_coo(np.random.randn(2,L))]+
#             [sparse.COO([idx1,idx2],np.random.randn(2*L),(2*L,L)).reshape((L,2,L))
#                for i in range(9)])
# #mps_line[0]

In [5]:
# L=10
# num=1
# k=2
# origin_inputs = rd_engine(L,num,k)
# inputs = origin_inputs.permute(1,2,0)#(B,num,k)->(num,k,B)
# inputs = torch.diag_embed(inputs)#(num,k,B)->(num,k,B,B)
# inputs = inputs.permute(0,2,1,3)#(num,k,B,B)->(num,B,k,B)
# num,B,k,B     = inputs.shape
# inputs        = inputs.reshape(num,B*k,B )

In [6]:
# import torch
# from torch_sparse import coalesce
# idx1          = list(range(2*L))
# idx2          = [i for i in range(L) for j in range(2)]
# index         = torch.tensor([idx1,idx2])
# sparse_tensor = torch.sparse_coo_tensor(index, origin_inputs.flatten(), (2*L,L)).coalesce()
# sparse_tensor = reshape_sparse_tensor(sparse_tensor,(L,2*L))

In [30]:
def transpose_sparse_tensor(coalesce_sparse_tensor):
    return coalesce_sparse_tensor.transpose(1,0).coalesce()

def matmul_sparse_tensor(coalesce_sparse_tensor_1,coalesce_sparse_tensor_2):
    m,k1   = coalesce_sparse_tensor_1.size()
    k2,n   = coalesce_sparse_tensor_2.size()
    assert k1==k2
    indexA = coalesce_sparse_tensor_1.indices()
    valueA = coalesce_sparse_tensor_1.values()
    
    indexB = coalesce_sparse_tensor_2.indices()
    valueB = coalesce_sparse_tensor_2.values()

    indexC, valueC = torch_sparse.spspmm(indexA, valueA, indexB, valueB, m, k1, n)
    return torch.sparse_coo_tensor(indexC, valueC, (m,n)).coalesce()
def reshape_sparse_tensor(coalesce_sparse_tensor,target_shape):
    indices = coalesce_sparse_tensor.indices().tolist()
    size    = coalesce_sparse_tensor.size()
    assert np.prod(size)==np.prod(target_shape)
    target_indices = np.stack(np.unravel_index(np.ravel_multi_index(indices,size),target_shape))
    target_indices = torch.Tensor(target_indices)
    target_indices = target_indices.to(coalesce_sparse_tensor.device)
    tensor = torch.sparse_coo_tensor(target_indices, coalesce_sparse_tensor.values(), target_shape).coalesce()
    #tensor.to(coalesce_sparse_tensor.device)
    return tensor

def sparse_diagonal(dense_matrix):
    A = len(dense_matrix)
    idx = torch.stack([torch.arange(A),torch.arange(A)]).to(dense_matrix.device)
    return torch.sparse_coo_tensor(idx,dense_matrix, (A,A)).coalesce()

def sparse_take_first(coalesce_sparse_tensor,row_num,axis=0):
    size    = list(coalesce_sparse_tensor.shape)
    if row_num > size[axis]:return coalesce_sparse_tensor
    indexes = coalesce_sparse_tensor.indices()
    value   = coalesce_sparse_tensor.values()
    good_i=indexes[axis]<row_num
    indexes = torch.stack([t[good_i] for t in indexes])
    value   = value[good_i]
    size[axis] = row_num
    return torch.sparse_coo_tensor(indexes,value, size).coalesce().to(coalesce_sparse_tensor.device)

def diagonal_tensor_svd_torch_sparse_2D(tensor):
    W,H   = tensor.size()
    if W>=H:
        MstarM     = matmul_sparse_tensor(transpose_sparse_tensor(tensor),tensor)
    else:
        MstarM     = matmul_sparse_tensor(tensor,transpose_sparse_tensor(tensor))
    diagonal_svd_flag = True
    a,b = MstarM.indices()
    if (a!=b).any():
        return None
    L,L = MstarM.shape
    batch_diag  = torch.Tensor([MstarM[i,i] for i in range(L)]).to(tensor.device)
    #batch_diag = MstarM.values()
    # in sparse representation, only value > 0 takes.
    batch_diag,batch_order = batch_diag.sort(-1,descending=True)
    s          = batch_diag.sqrt()
    nonzero_num= torch.sum(s>0)
    s          = s[:nonzero_num]
    A = len(batch_order)
    index = torch.stack([torch.arange(A).to(tensor.device),batch_order])
    value = torch.ones(A).to(tensor.device)
    if W>=H: 
        v = torch.sparse_coo_tensor(index,value,(A,A)).coalesce()
        if nonzero_num < A :v = sparse_take_first(v,nonzero_num,axis=1)
        u = matmul_sparse_tensor(tensor,matmul_sparse_tensor(transpose_sparse_tensor(v),sparse_diagonal(1/s)))
    else:
        u = torch.sparse_coo_tensor(index,value,(A,A)).coalesce()
        if nonzero_num < A :u = sparse_take_first(u,nonzero_num,axis=0)
        v = matmul_sparse_tensor(matmul_sparse_tensor(sparse_diagonal(1/s),u),tensor)
        u = transpose_sparse_tensor(u)
    return u,s,v

def reciprocal_sparse_tensor(coalesce_sparse_tensor):
    size   = coalesce_sparse_tensor.size()
    indexA = coalesce_sparse_tensor.indices()
    valueA = coalesce_sparse_tensor.values()
    return torch.sparse_coo_tensor(indexA, 1/valueA, size).coalesce()

def diagonal_tensor_RQ_torch_sparse(tensor):
    W,H   = tensor.shape
    assert W<=H
    MstarM     = matmul_sparse_tensor(tensor,transpose_sparse_tensor(tensor))
    a,b = MstarM.indices()
    assert (a==b).any()
    #batch_diag = MstarM.values()
    #s = batch_diag.sqrt()
    R = MstarM.sqrt()
    Q = matmul_sparse_tensor(reciprocal_sparse_tensor(R),tensor)
    return R,Q

def right_canonicalize_MPS_torch_sparse(mps_line,Decomposition_Engine=diagonal_tensor_RQ_torch_sparse,
                          final_normlization =True,all_renormlization=False
                          ):
    new_chain = []
    R         = None
    Z_list    = []
    # assume every mps_unit is store (kD,D)
    for i,tensor in enumerate(mps_line[::-1]):
        if R is not None:
            new_tensor = matmul_sparse_tensor(tensor,R)   
        else:
            new_tensor = tensor
        kD,D  = new_tensor.shape
        if kD>D:
            new_tensor = reshape_sparse_tensor(new_tensor,(D,kD))

        if i == len(mps_line) - 1:
            if final_normlization:
                Z = (new_tensor**2).values().sum().sqrt()
                new_tensor /= Z
                Z_list.append(Z)
            new_chain.append(new_tensor)
        else:
            if all_renormlization:
                Z = (new_tensor**2).values().sum().sqrt()
                new_tensor /= Z
                Z_list.append(Z)
            R,Q = Decomposition_Engine(new_tensor)[:2]
            new_chain.append(Q)
    new_chain=new_chain[::-1]
    return new_chain,Z_list

def left_canonicalize_MPS_torch_sparse(mps_line,Decomposition_Engine=None,
                          final_normlization =True,all_renormlization=False):
    # for any not canonical mps line
    # the chain size (D,P,D)
    new_chain = []
    R         = None
    Z_list    = []# record the scale information for each unit.
    # assume every mps_unit is store (D,kD)
    for i,tensor in enumerate(tqdm(mps_line)):
        if R is not None:
            D,kD       = tensor.shape
            B,D        = R.shape
            new_tensor = matmul_sparse_tensor(R,tensor)   
            new_shape  = new_tensor.shape
            new_tensor = reshape_sparse_tensor(new_tensor,(B*kD//D,D))
        else:
            new_tensor = tensor        
        if i == len(mps_line) - 1:
            if final_normlization:
                Z = (new_tensor**2).values().sum().sqrt()
                new_tensor /= Z
                Z_list.append(Z)
            new_chain.append(new_tensor)
        else:
            if all_renormlization:
                Z = (new_tensor**2).values().sum().sqrt()
                new_tensor /= Z
                Z_list.append(Z)
            Q,R,_,diagonal_svd_flag = Decomposition_Engine(new_tensor)
            new_chain.append(Q)
           # print(Q.shape)
#         if not diagonal_svd_flag:
#             print(f"full matrix SVD at unit {i}")

    return new_chain,Z_list

def truncated_SVD_torch_sparse(tensor,output='RQ',max_singular_values= None,
                          max_truncation_error= None,
                          relative = True,
                          normlized= True,
                          verbose  = False):
    # the canonocal
    # tensor is batched
    assert len(tensor.shape)  == 2
    diagonal_svd_flag = True
    out = diagonal_tensor_svd_torch_sparse_2D(tensor)
    if out is None:
        diagonal_svd_flag=False
        q = min(tensor.size())
        q = min(q,max_singular_values) 
        u, s, v = torch.svd_lowrank(tensor,q=q)
        v       = v.T
        #print(q,s.shape)
    else:
        u, s, v = out
        max_singular_values = s.shape[-1] if max_singular_values is None else max_singular_values
        if max_truncation_error is not None and len(s)>max_singular_values:
            # Cumulative norms of singular values in ascending order
            s_normlized  = s**2
            s_normlized /= torch.sum(s_normlized)
            s_sorted,_   = torch.sort(s_normlized)# 0.1,0.2,...,0.4
            trunc_errs   = torch.sqrt(torch.cumsum(s_sorted,0))# 0.1,0.3,....1
            # If relative is true, rescale max_truncation error with the largest
            # singular value to yield the absolute maximal truncation error.
            num_sing_vals_err = torch.sum(trunc_errs > max_truncation_error)
            if max_singular_values>num_sing_vals_err and verbose:
                print(f"use {num_sing_vals_err}/{max_singular_values} sing vals")
        else:
            num_sing_vals_err  = max_singular_values

        nk = min(max_singular_values, num_sing_vals_err)


        #s_rest = s[...,num_sing_vals_keep:]

        u  = sparse_take_first(u,nk,axis=1) if u.is_sparse else u[...,:nk]
        s  = s[...,:nk]
        v  = sparse_take_first(v,nk,axis=0) if v.is_sparse else v[:nk,:]
    Z  = None #maybe add in the furture
    if output == 'RQ':
        R = u*s[None] if not u.is_sparse else matmul_sparse_tensor(u,sparse_diagonal(s))
        Q = v
        if not R.is_sparse:R = R.to_sparse() 
        if not Q.is_sparse:Q = Q.to_sparse() 
        output = [R,Q,Z,diagonal_svd_flag]
    elif output == 'QR':
        Q = u
        R = s[:,None]*v if not  v.is_sparse else matmul_sparse_tensor(sparse_diagonal(s),v)
        if not R.is_sparse:R = R.to_sparse()
        if not Q.is_sparse:Q = Q.to_sparse()
        output = [Q,R,Z,diagonal_svd_flag]
    else:
        if not u.is_sparse:u = u.to_sparse() 
        if not v.is_sparse:v = v.to_sparse() 
        output = [u,s,v,Z,diagonal_svd_flag]
    return output

In [7]:
train_loader= torch.utils.data.DataLoader(dataset=mnist_train, batch_size=1000, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=mnist_test , batch_size=1000, shuffle=False)
images,labels = iter(train_loader).next()
origin_inputs = preprocess_sum_one(images)

L=origin_inputs.shape[0]
idx1          = list(range(2*L))
idx2          = [i for i in range(L) for j in range(2)]
index         = torch.tensor([idx1,idx2])
mps_line=[(origin_inputs[:,0,:].transpose(1,0)).to_sparse()]
for tensor in origin_inputs.permute(1,0,2)[1:]:
    sparse_tensor = torch.sparse_coo_tensor(index, tensor.flatten(), (2*L,L)).coalesce()      
    #sparse_tensor = reshape_sparse_tensor(sparse_tensor,(L,2*L))                              
    mps_line.append(sparse_tensor)

In [11]:
mps_line=[t.cuda() for t in mps_line]

In [21]:
import torch_sparse

In [22]:
mps_line,Z = right_canonicalize_MPS_torch_sparse(mps_line,final_normlization =True,all_renormlization=False)

In [25]:
from tqdm.notebook import tqdm

In [38]:
Decomposition_Engine=lambda x:truncated_SVD_torch_sparse(x,output='QR',
                                                        max_truncation_error=0.00,
                                                        max_singular_values=1,
                                                       )
mps_line2,z_2 = left_canonicalize_MPS_torch_sparse(mps_line,final_normlization =True,
                                               all_renormlization=True,
                                               Decomposition_Engine=Decomposition_Engine)

  0%|          | 0/784 [00:00<?, ?it/s]

###### torch_sparse MPS

In [1]:
import torch
import torch_sparse
import numpy as np
from utils import *

def rd_engine(*x,**kargs):
    x =  torch.randn(*x,device='cpu',**kargs)
    x/=  torch.norm(x).sqrt()
    return x

In [2]:
from torchvision import datasets, transforms
mnist_data = np.load('archive/tn-for-unsup-ml/data/binarized_mnist.npz')
train_data = torch.from_numpy(mnist_data['train_data'])
test_data  = torch.from_numpy(mnist_data['test_data'])

transform = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Normalize(mean=(0.0,), std=(1.0,))
])
DATAPATH    = '/media/tianning/DATA/DATASET/MNIST/'
mnist_train = datasets.MNIST(DATAPATH, train=True, download=False, transform=transform)
mnist_test  = datasets.MNIST(DATAPATH, train=False,download=False, transform=transform)


In [17]:
train_loader= torch.utils.data.DataLoader(dataset=mnist_train, batch_size=1000, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=mnist_test , batch_size=1000, shuffle=False)
images,labels = iter(train_loader).next()
origin_inputs = preprocess_sum_one(images)

In [18]:
origin_inputs.shape

torch.Size([1000, 784, 2])

In [3]:
device='cpu'
L=origin_inputs.shape[0]
idx1  = [0]*L+[1]*L
idx2  = [(L+1)*i for i in range(L)]
idx2  = idx2 + idx2
index = torch.tensor([idx2,idx1])
input_mps=[(origin_inputs[:,0,:]).to_sparse().to(device)]
for tensor in origin_inputs.permute(1,0,2)[1:]:
    sparse_tensor = torch.sparse_coo_tensor(index, tensor.flatten(), (L*L,2)).coalesce()      
    #sparse_tensor = reshape_sparse_tensor(sparse_tensor,(L,2*L))                              
    input_mps.append(sparse_tensor.to(device))
    

In [ ]:
# left_tensors = [torch.einsum('cpd,apb->acbd', input_data[i]  ,self.mps_var[i]).flatten(0,1).flatten(-2,-1) for i in range(self.hn)]
# rigt_tensors = [torch.einsum('cpd,apb->acbd', input_data[i-1]  ,self.mps_var[i]).flatten(0,1).flatten(-2,-1) for i in range(self.hn+1,len(self.mps_var))]


In [38]:
D=2
P=2
L=14
B=3
inputs = rd_engine(B,L,P)    
# inputs = inputs.permute(1,2,0)#(B,num,k)->(num,k,B)
# inputs = torch.diag_embed(inputs)#(num,k,B)->(num,k,B,B)
# inputs = inputs.flatten(-2,-1)
#inputs = inputs.permute(0,2,1,3)#(num,k,B,B)->(num,B,k,B)

In [39]:
B=inputs.shape[0]
idx1  = [0]*B+[1]*B
idx2  = [(B+1)*i for i in range(B)]
idx2  = idx2 + idx2
index = torch.tensor([idx2,idx1])
input_mps=[(inputs[:,0,:]).to_sparse()]
for tensor in inputs.permute(1,0,2)[1:]:
    sparse_tensor = torch.sparse_coo_tensor(index, tensor.flatten(), (B*B,2)).coalesce()      
    #sparse_tensor = reshape_sparse_tensor(sparse_tensor,(L,2*L))                              
    input_mps.append(sparse_tensor)

In [70]:

def rd_engine(*x,**kargs):
    x =  torch.randn(*x,device='cpu',**kargs)
    x/=  torch.norm(x).sqrt()
    return x
Ds  = [1]+list(np.random.randint(3,10,L-1))+[1]
mps_var    = [rd_engine(Ds[i],P,Ds[i+1]) for i in range(L)]  
print(Ds)

In [75]:
Ds

[1, 3, 4, 9, 6, 4, 6, 6, 3, 6, 5, 6, 6, 3, 1]

In [85]:
out=[]
for inp,mps_unit in zip(input_mps,mps_var):
    out.append(torch.sparse.mm(inp,mps_unit.permute(1,0,2).flatten(1,2)))

In [81]:
idx=4
print(input_mps[idx].shape)
D1,P,D2=mps_var[idx].shape

torch.Size([9, 2])


In [78]:
all_values= input_mps[idx].values().reshape(B,P)@mps_var[idx].permute(1,0,2).flatten(1,2)

In [80]:
all_values.shape

torch.Size([3, 24])

In [84]:
torch.block_diag(*all_values.reshape(B,D1,D2))

tensor([[ 0.0746, -0.0109,  0.0053,  0.0967,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0675, -0.0708,  0.1573, -0.1155,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0360,  0.2245,  0.0516,  0.1981,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.2776, -0.2080, -0.0120,  0.1236,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.1618,  0.0482,  0.0096, -0.0264,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.1465, -0.1226, -0.1381,  0.1770,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.1027, -0.0411, -0.0813,  0.1604,
          0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.1903, -0.0082,  0.2638, -0.0855,
          0.0000,  0.0000,  0.

In [89]:
LDLD=out[idx].reshape(B,B,Ds[idx],Ds[idx+1]).permute(0,2,1,3).flatten(0,1).flatten(-2,-1).to_sparse()

In [93]:
LDLD.shape

torch.Size([18, 12])

In [92]:
print(LDLD.to_dense().numpy().round(3))

[[ 0.075 -0.011  0.005  0.097  0.     0.     0.     0.     0.     0.
   0.     0.   ]
 [ 0.067 -0.071  0.157 -0.115  0.     0.     0.     0.     0.     0.
   0.     0.   ]
 [-0.036  0.225  0.052  0.198  0.     0.     0.     0.     0.     0.
   0.     0.   ]
 [ 0.278 -0.208 -0.012  0.124  0.     0.     0.     0.     0.     0.
   0.     0.   ]
 [ 0.162  0.048  0.01  -0.026  0.     0.     0.     0.     0.     0.
   0.     0.   ]
 [ 0.146 -0.123 -0.138  0.177  0.     0.     0.     0.     0.     0.
   0.     0.   ]
 [ 0.     0.     0.     0.     0.103 -0.041 -0.081  0.16   0.     0.
   0.     0.   ]
 [ 0.     0.     0.     0.     0.19  -0.008  0.264 -0.086  0.     0.
   0.     0.   ]
 [ 0.     0.     0.     0.     0.05   0.324  0.047  0.228  0.     0.
   0.     0.   ]
 [ 0.     0.     0.     0.     0.446 -0.231 -0.047  0.256  0.     0.
   0.     0.   ]
 [ 0.     0.     0.     0.     0.258  0.031  0.023 -0.027  0.     0.
   0.     0.   ]
 [ 0.     0.     0.     0.     0.195 -0.238 -0.238  0.

###### test

the max singular value truncation only good for hundreds dimenstion, for small dimenstion not good

FiniteMPS canonicalize == right_canonicalize_MPS with torchrq

In [3]:
import torch

In [2]:
# from tensornetwork.matrixproductstates.finite_mps import FiniteMPS
# from tensornetwork.matrixproductstates.infinite_mps import InfiniteMPS
# from typing import Any, List, Optional, Text, Type, Union, Dict, Sequence
# import numpy as np
# Tensor = Any

In [10]:
# D=6
# P=4
# L=5
# mps_line = [torch.randn(1,P,D)] + [torch.randn(D,P,D) for i in range(L-2)]+ [torch.randn(D,P,1)]
# input_mps= [torch.randn(P,D)] + [torch.randn(D,P,D) for i in range(L-2)]+ [torch.randn(D,P)]
# approx = approxmate_mps_line(mps_line,max_singular_values=2)
# #contract_two_mps(approx,input_mps)

In [16]:
# tn.set_default_backend("pytorch")
# tn_mps_1 = FiniteMPS(mps_line,canonicalize=False)
# tn_mps_1.canonicalize(normalize=False)
# tn_mps_1.center_position

In [4]:
import tensornetwork as tn
from tensornetwork import contractors
tn.set_default_backend("pytorch")

In [4]:
from utils import *

In [5]:
D=10
P=4
L=14
def rd_engine(*x,**kargs):
    x =  torch.randn(*x,device='cpu',**kargs)
    x/=  torch.norm(x).sqrt()
    return x

In [6]:
from tensornetwork.contractors.opt_einsum_paths.path_contractors import *
def my_contracter(nodes: Iterable[AbstractNode],
         path = None,
         output_edge_order: Optional[Sequence[Edge]] = None,
         ignore_edge_order: bool = False,
         memory_limit: Optional[int] = None) -> AbstractNode:
        """Base method for all `opt_einsum` contractors.

        Args:
        nodes: A collection of connected nodes.
        algorithm: `opt_einsum` contraction method to use.
        output_edge_order: An optional list of edges. Edges of the
        final node in `nodes_set`
        are reordered into `output_edge_order`;
        if final node has more than one edge,
        `output_edge_order` must be provided.
        ignore_edge_order: An option to ignore the output edge
        order.

        Returns:
        Final node after full contraction.
        """
        nodes_set = set(nodes)
        edges = get_all_edges(nodes_set)
        #output edge order has to be determinded before any contraction
        #(edges are refreshed after contractions)

        if not ignore_edge_order:
            if output_edge_order is None:
                output_edge_order = list(get_subgraph_dangling(nodes))
                if len(output_edge_order) > 1:
                    raise ValueError("The final node after contraction has more than "
                                 "one remaining edge. In this case `output_edge_order` "
                                 "has to be provided.")

            if set(output_edge_order) != get_subgraph_dangling(nodes):
                raise ValueError("output edges are not equal to the remaining "
                       "non-contracted edges of the final node.")

        for edge in edges:
            if not edge.is_disabled:  #if its disabled we already contracted it
                if edge.is_trace():
                    nodes_set.remove(edge.node1)
                    nodes_set.add(contract_parallel(edge))

        if len(nodes_set) == 1:
            # There's nothing to contract.
            if ignore_edge_order:
                return list(nodes_set)[0]
            return list(nodes_set)[0].reorder_edges(output_edge_order)

        if path is None:
            algorithm = functools.partial(opt_einsum.paths.greedy, memory_limit=memory_limit)
            # Then apply `opt_einsum`'s algorithm
            path, nodes = utils.get_path(nodes_set, algorithm)
        else:
            nodes = list(nodes_set) 
        for a, b in path:
            new_node = contract_between(nodes[a], nodes[b], allow_outer_product=True)
            nodes.append(new_node)
            nodes = utils.multi_remove(nodes, [a, b])

        # if the final node has more than one edge,
        # output_edge_order has to be specified
        final_node = nodes[0]  # nodes were connected, we checked this
        if not ignore_edge_order:
            final_node.reorder_edges(output_edge_order)
        return final_node,path

In [7]:
D=10;O=10;mps_core = [rd_engine(P,D)] + [rd_engine(D,P,D) for _ in range(L-1)] + [rd_engine(D,O,D)]+ [rd_engine(D,P)]
B=20;inp_line = [rd_engine(B,P)] + [rd_engine(B,P,B) for _ in range(L-1)] + [rd_engine(B,P,B)]

In [33]:
mps_nodes  = [tn.Node(v, name=f"t{i}") for i,v in enumerate(mps_core)]
for i in range(len(mps_core)-1):
    tn.connect(mps_nodes[i][-1],mps_nodes[i+1][0],name=f"mps:{i}<->{i+1}")
inp_nodes=[tn.Node(v, name=f"i{i}") for i,v in enumerate(inp_line)]
tn.connect(inp_nodes[0][0],inp_nodes[1][0],name=f"inp:{0}<->{1}")
for i in range(1,len(inp_nodes)-1):
    tn.connect(inp_nodes[i][-1],inp_nodes[i+1][0],name=f"inp:{i}<->{i+1}")
for i,input_node in enumerate(inp_nodes):
    j = i if i < L else i+1
    mps_physicd_edge = mps_nodes[j][0] if j==0 else mps_nodes[j][1]
    inp_physics_edge = input_node[1]
    tn.connect(mps_physicd_edge,inp_physics_edge,name=f"phy_{i}")
ans,path = my_contracter(mps_nodes+inp_nodes,
                       output_edge_order=[inp_nodes[-1][2],mps_nodes[L][1]])
ans.tensor

[(7, 18), (12, 28), (10, 27), (14, 17), (25, 26), (0, 25), (21, 24), (20, 23), (19, 22), (5, 21), (1, 20), (1, 19), (1, 18), (8, 17), (11, 16), (12, 15), (5, 14), (4, 13), (7, 12), (7, 11), (8, 10), (4, 9), (0, 8), (2, 7), (1, 6), (0, 5), (0, 4), (0, 3), (0, 2), (0, 1)]


tensor([[-1.4304e-01, -1.5820e-01, -1.3586e-02, -9.4462e-04, -1.3982e-01,
          9.9057e-02,  1.7996e-01, -1.1456e-01,  1.4263e-01, -2.4580e-03],
        [ 3.4687e-01, -4.8310e-01,  2.2043e-01, -2.1364e-01,  4.1445e-01,
         -2.5674e-01,  1.4035e-01,  2.4091e-01, -2.3025e-01, -1.1829e-01],
        [ 2.0311e-01, -8.6519e-02,  3.1690e-01,  2.3554e-01,  1.1007e-02,
         -4.1188e-01,  4.2258e-02,  3.6463e-01, -3.4286e-01, -4.0262e-01],
        [-4.7547e-01,  3.7831e-01,  3.3061e-02,  2.4604e-01, -1.5482e-01,
         -4.6528e-02,  4.5324e-03, -4.0088e-01,  3.5013e-01,  8.5555e-02],
        [ 9.9050e-02, -8.2526e-02,  1.8456e-01, -1.9140e-02,  1.1645e-01,
         -3.5684e-01, -2.0432e-01,  1.1110e-01,  9.2640e-02, -1.0480e-01],
        [ 1.6438e-01,  2.7121e-01, -3.3816e-02, -4.1494e-03,  1.2835e-01,
          1.2292e-01,  8.6925e-02,  3.1019e-01,  3.8543e-01,  1.7188e-01],
        [-5.1561e-02,  1.1307e-01, -5.6854e-01,  1.4294e-01, -2.3753e-01,
          2.2774e-01,  3.7236e-0

In [ ]:
top_mps_list    = [rd_engine(P,D)] + [rd_engine(L-2,D,P,D)]  + [rd_engine(D,P)]
middle_mpo_list =[[rd_engine(P,D,P)]+[rd_engine(L-2,D,P,D,P)]+ [rd_engine(D,P,P)]
                  for _ in range(L-2)]
bottom_mps_list = [rd_engine(P,D)] + [rd_engine(L-2,D,P,D)]  + [rd_engine(D,P)]

In [18]:
from utils_contraction_tool import *

In [13]:
D=2000
P=4
L=14
top_mps_list = [rd_engine(P,D)] + [rd_engine(L-2,D,P,D)]  + [rd_engine(D,P)]
top_mps_list = right_mps_form(top_mps_list)
#apply some noise.
bottom_mps_list= [0.005*torch.randn_like(a)+a for a in top_mps_list]

In [14]:
bottom_mps_list,Zb = left_canonicalize_MPS(right_mps_form(bottom_mps_list))

In [ ]:
top_mps_list

In [21]:
contract_two_mps(bottom_mps_list,top_mps_list)

NameError: name 'contract_two_mps' is not defined

In [34]:
new_mps_list, scalar = approxmate_mps_line(top_mps_list,max_singular_values=1000)
contract_two_mps(bottom_mps_list,new_mps_list)*scalar

   left canonical scalar:0.09317030757665634
   right canonical Z:[1.0, 1.0, 1.0, 1.0, 1.0, 0.999, 0.942, 0.938, 0.958, 1.0, 1.0, 1.0, 1.0, 1.0]
   right canonical scalar:0.8452922105789185


tensor(0.0118)

In [7]:
now_mps_list = bottom_mps_list
print(f"start:0/{len(middle_mpo_list)} {get_mps_size_list(now_mps_list)}")
for i,next_mpo in enumerate(middle_mpo_list):
    
    start_time = time.time()
    now_mps_list = contract_mps_mpo(now_mps_list,next_mpo)
    print(f"start:{i+1}/{len(middle_mpo_list)} ")
    print(f"get mps {get_mps_size_list(now_mps_list)}")
    cost       = time.time() - start_time
    print(f"boundary contraction cost:{cost}")
    if now_mps_list[0].shape[-1]>1:
        start_time = time.time()
        now_mps_list,scalar = approxmate_mps_line(right_mps_form(now_mps_list),
                                                  max_singular_values= 1000,
                                                  mode='full',
                                                 # mode='right' if i>3 else 'full'
                                                 )
        #now_mps_list = [now_mps_list[0],torch.stack(now_mps_list[1:-1],dim=0),now_mps_list[-1]]
        cost       = time.time() - start_time
        print(f"low rank approximation cost:{cost} scalar:{scalar}")
        print("==========================================")
#     start_time = time.time()
#     value      = contract_two_mps(now_mps_list,top_mps_list)
#     cost       = time.time() - start_time
#     print(f"col contraction cost:{cost}")

start:0/12 (4, 6) - 12x(6, 4, 6) - (6, 4)
start:1/12 
get mps (4, 36) - 12x(36, 4, 36) - (36, 4)
boundary contraction cost:0.0013422966003417969
   left canonical scalar:3.0327204513014294e-05
   right canonical Z:[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
   right canonical scalar:1.0
low rank approximation cost:0.11043429374694824 scalar:3.0327204513014294e-05
start:2/12 
get mps (4, 24)-(24, 4, 96)-(96, 4, 216)-(216, 4, 216)-(216, 4, 216)-(216, 4, 216)-(216, 4, 216)-(216, 4, 216)-(216, 4, 216)-(216, 4, 216)-(216, 4, 216)-(216, 4, 96)-(96, 4, 24)-(24, 4)
boundary contraction cost:0.0018436908721923828
   left canonical scalar:0.000614150136243552
   right canonical Z:[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
   right canonical scalar:1.0000004768371582
low rank approximation cost:0.23517894744873047 scalar:0.0006141504272818565
start:3/12 
get mps (4, 24)-(24, 4, 96)-(96, 4, 384)-(384, 4, 1296)-(1296, 4, 1296)-(1296, 4, 1296)-(

In [45]:
now_mps_list = bottom_mps_list
for i,next_mpo in enumerate(middle_mpo_list):
    
    start_time = time.time()
    now_mps_list = contract_mps_mpo(now_mps_list,next_mpo)
    print(f"start:{i}/{len(middle_mpo_list)} {get_mps_size_list(now_mps_list)}")
    cost       = time.time() - start_time
    print(f"boundary contraction cost:{cost}")
    if now_mps_list[0].shape[-1]>1:
        start_time = time.time()
        now_mps_list,scalar = approxmate_mps_line(right_mps_form(now_mps_list),max_singular_values= 1000)
        #now_mps_list = [now_mps_list[0],torch.stack(now_mps_list[1:-1],dim=0),now_mps_list[-1]]
        cost       = time.time() - start_time
        print(f"low rank approximation cost:{cost} scalar:{scalar}")
#     start_time = time.time()
#     value      = contract_two_mps(now_mps_list,top_mps_list)
#     cost       = time.time() - start_time
#     print(f"col contraction cost:{cost}")

start:0/10 (4, 36) - 10x(36, 4, 36) - (36, 4)
boundary contraction cost:0.0007288455963134766
low rank approximation cost:0.010253667831420898 scalar:5.410343841081405e-23
start:1/10 (4, 24)-(24, 4, 96)-(96, 4, 216)-(216, 4, 216)-(216, 4, 216)-(216, 4, 216)-(216, 4, 216)-(216, 4, 216)-(216, 4, 216)-(216, 4, 96)-(96, 4, 24)-(24, 4)
boundary contraction cost:0.0014865398406982422
low rank approximation cost:0.038602352142333984 scalar:1.2635199960961407e-13
start:2/10 (4, 24)-(24, 4, 96)-(96, 4, 384)-(384, 4, 1296)-(1296, 4, 1296)-(1296, 4, 1296)-(1296, 4, 1296)-(1296, 4, 1296)-(1296, 4, 384)-(384, 4, 96)-(96, 4, 24)-(24, 4)
boundary contraction cost:0.03164219856262207
low rank approximation cost:0.9437947273254395 scalar:1.3323970424988897e-13
start:3/10 (4, 24)-(24, 4, 96)-(96, 4, 384)-(384, 4, 1536)-(1536, 4, 6000)-(6000, 4, 6000)-(6000, 4, 6000)-(6000, 4, 1536)-(1536, 4, 384)-(384, 4, 96)-(96, 4, 24)-(24, 4)
boundary contraction cost:0.4210662841796875
low rank approximation cost:16

KeyboardInterrupt: 

In [31]:
[t.shape for t in now_mps_list[1:-1]]

[torch.Size([4, 4, 16]),
 torch.Size([16, 4, 64]),
 torch.Size([64, 4, 256]),
 torch.Size([256, 4, 64]),
 torch.Size([64, 4, 16]),
 torch.Size([16, 4, 4])]

In [ ]:
def contraction_row(tensor1,tensor2,truncate=None,einsum_engin=einsum_engin):
    # tensor1 <-> tensor2 :-> tensor
    # (N,H,a,b,c,d) <-> (N,H,c,e,f,g) :-> (N,H,a,be,f,dg)
    tensor  = einsum_engin("whabcd,whcefg->whabefdg",tensor1,tensor2).flatten(3,4).flatten(-2,-1)
    if truncate is None:return tensor 
    W,H = tensor.shape[:2]
    

In [ ]:
def RecursionBMPS(tensor,truncate=None,einsum_engin=torch.einsum):
    W,H = tensor.shape[:2]
    while W > 1:
            half_size = size // 2
            nice_size = 2 * half_size
            leftover  = tensor[nice_size:]
            tensor    = contraction_line(tensor[0:nice_size:2], tensor[1:nice_size:2],truncate=None,einsum_engin=einsum_engin)
            #tensor    = torch.einsum("mbik,mbkj->mbij",tensor[0:nice_size:2], tensor[1:nice_size:2])
            #(k/2,NB,D,D),(k/2,NB,D,D) <-> (k/2,NB,D,D)
            tensor   = torch.cat([tensor, leftover], axis=0)
            size     = half_size + int(size % 2 == 1)
    return value